In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler

# Download dataset from Kaggle
!kaggle datasets download -d phucthaiv02/butterfly-image-classification -p /content/
import zipfile

# Unzip the dataset zip file
with zipfile.ZipFile('/content/butterfly-image-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/butterfly_image_dataset')

# Load the CSV with class data
csv_path = '/content/butterfly_image_dataset/Training_set.csv'
df = pd.read_csv(csv_path)

# Selecting labels and creating the class dictionary
selected_labels = df['label'].unique()[:10]  # Limit to 10 labels for demonstration
class_dict = df[df['label'].isin(selected_labels)].groupby('label')['filename'].apply(lambda x: x.head(30).tolist()).to_dict()

# Convert the class dictionary to a DataFrame and save as CSV
class_dict_df = pd.DataFrame([(label, filename) for label, filenames in class_dict.items() for filename in filenames], columns=['label', 'filename'])
csv_output_path = '/content/butterfly_image_dataset/picked_classes.csv'
class_dict_df.to_csv(csv_output_path, index=False)

# Load the CSV
loaded_df = pd.read_csv(csv_output_path)

# Convert to a dictionary format like the original code
picked_classes = loaded_df.groupby('label')['filename'].apply(list).to_dict()

# Flatten the filenames for all images
all_image_filenames = np.array(list(picked_classes.values()))
print("Shape of image filenames:", all_image_filenames.shape)

Dataset URL: https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
License(s): CC0-1.0
 97% 220M/226M [00:03<00:00, 89.0MB/s]
100% 226M/226M [00:03<00:00, 72.3MB/s]
Shape of image filenames: (10, 30)


In [ ]:
# Image loading function
images_folder = '/content/butterfly_image_dataset/train'

def load_and_process_image(image_name):
    img = cv2.imread(os.path.join(images_folder, image_name))
    if img is None:
        print(f"Failed to load image: {image_name}")
        return None
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = np.mean(img, axis=2).astype(np.uint8)
    return img_gray

# Process images for each class
image_matrices = []

for class_images in all_image_filenames:
    class_image_matrices = []
    for image_name in class_images:
        img_matrix = load_and_process_image(image_name)
        if img_matrix is not None:
            class_image_matrices.append(img_matrix)
    image_matrices.append(class_image_matrices)

image_matrices = np.array(image_matrices, dtype=np.uint8)
print("Shape of all image matrices:", image_matrices.shape)



Shape of all image matrices: (10, 30, 256, 256)


In [ ]:
# Function to extract top-left corner of an image matrix
def get_top_left_corner(matrix, n):
    return matrix[:n, :n]

# Preparing dataset with the top-left corner extraction
rows, columns = 10, 30  # As per the number of labels and images
dataset = np.empty((rows, columns, 2, 2), dtype=image_matrices[0][0].dtype)

for i in range(rows):
    for j in range(columns):
        dataset[i, j] = get_top_left_corner(image_matrices[i][j], 2)

print(dataset.shape, dataset)

(10, 30, 2, 2) [[[[127 131]
   [123 127]]

  [[255 255]
   [255 255]]

  [[ 15  15]
   [ 14  14]]

  ...

  [[ 37  38]
   [ 35  32]]

  [[ 43  44]
   [ 42  44]]

  [[ 66  66]
   [ 65  65]]]


 [[[ 55  56]
   [ 56  56]]

  [[ 84  80]
   [ 86  85]]

  [[ 94  95]
   [ 95  95]]

  ...

  [[109 109]
   [107 108]]

  [[ 16  15]
   [ 20  18]]

  [[ 58  55]
   [ 56  54]]]


 [[[232 232]
   [232 232]]

  [[ 69  67]
   [ 70  69]]

  [[ 72 163]
   [ 61 156]]

  ...

  [[126 130]
   [134 132]]

  [[ 50  51]
   [ 51  51]]

  [[138 138]
   [139 138]]]


 ...


 [[[228 230]
   [230 233]]

  [[170 165]
   [169 163]]

  [[ 63  62]
   [ 63  61]]

  ...

  [[255 255]
   [255 255]]

  [[121 117]
   [118 115]]

  [[ 87  87]
   [ 86  87]]]


 [[[ 85  86]
   [ 85  87]]

  [[ 89  96]
   [ 89  93]]

  [[125 126]
   [126 126]]

  ...

  [[255 255]
   [255 255]]

  [[ 56  53]
   [ 58  54]]

  [[ 66  68]
   [ 67  68]]]


 [[[ 33  33]
   [ 34  32]]

  [[233 232]
   [225 231]]

  [[ 40  39]
   [ 39  39]]

  ...

  

In [ ]:
# Function to separate predictors and target
def split_predictors_and_target(dataset):
    predictors = []
    targets = []
    for img_matrix in dataset:
        img_matrix_flat = img_matrix.flatten()
        predictors.append(img_matrix_flat[:-1])  # all but the last value as predictor
        targets.append(img_matrix_flat[-1])  # last value as target
    return np.array(predictors), np.array(targets)

# Train-Test Split Function
def split_train_test(X, y, Test_Num):
    X_Test = np.copy(X[Test_Num])
    y_Test = np.copy(y[Test_Num])
    X_Train = np.concatenate((X[:Test_Num], X[Test_Num + 1:]), axis=0)
    y_Train = np.concatenate((y[:Test_Num], y[Test_Num + 1:]), axis=0)
    return X_Train, y_Train, X_Test, y_Test

# Build and compile the ANN model
model = Sequential()
model.add(Input(shape=(3,)))
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

MAPE = []
average_MAPE = []
n = 30  # Number of images per label

# Train and predict for each row in the dataset
for i in range(rows):
    print(i)
    predictors, targets = split_predictors_and_target(dataset[i])

    # Scaling predictors and targets
    predictors_scaler = StandardScaler()
    targets_scaler = StandardScaler()

    fitted_predictors_scaler = predictors_scaler.fit(predictors)
    scaled_predictors = fitted_predictors_scaler.transform(predictors)

    reshaped_targets = targets.reshape(-1, 1)
    fitted_targets_scaler = targets_scaler.fit(reshaped_targets)
    scaled_targets = fitted_targets_scaler.transform(reshaped_targets)

    for j in range(n):
        X_Train, y_Train, X_Test, y_Test = split_train_test(
            scaled_predictors, scaled_targets, n - 1 - j
        )

        indices = list(range(n))
        indices.remove(n - 1 - j)
        print(j, indices)

        model.fit(X_Train, y_Train, batch_size=1, epochs=10, verbose=1)

        X_Test = X_Test.reshape(1, -1)
        predictions = model.predict(X_Test)

        predictions = fitted_targets_scaler.inverse_transform(predictions)

        y_Test = y_Test.reshape(-1, 1)
        y_Test = fitted_targets_scaler.inverse_transform(y_Test)

        MAPE.append(abs((y_Test - predictions)) / y_Test * 100)

    average_MAPE.append(sum(MAPE) / len(MAPE))

# Create a DataFrame with class labels and their corresponding MAPE values
class_labels = selected_labels[:10]
mape_results = [(class_labels[i], average_MAPE[i]) for i in range(10)]

# Create a DataFrame to display the results
mape_df = pd.DataFrame(mape_results, columns=['Class', 'Average MAPE'])

# Print the DataFrame to see the output
print(mape_df)

Streaming output truncated to the last 5000 lines.
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012     
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0021 
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4244e-04 
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0020 
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1527e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
14 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0021     
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0033     
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5217e-04  
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.7020e-04
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013     
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0022 
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - lo